In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Tugas Akhir/dataset/'

/content/drive/MyDrive/Tugas Akhir/dataset


In [3]:
!pip install keras-metrics
!pip install dataframe-image
!pip install varname
!pip install h5py
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 37.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 5.6 MB/s eta 0:00:00


In [4]:
import nltk
import pandas as pd
import re
import tensorflow as tf
import time
import multiprocessing
import io
import gensim
import numpy as np
import matplotlib.pyplot as plt
import keras_metrics as km
import pickle
import keras


from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
#from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Embedding, LSTM, Bidirectional, Dropout, GRU
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from gensim.models import word2vec
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from varname import nameof


nltk.download('punkt')
from nltk.tokenize import word_tokenize


from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from pickle import dump
from pickle import load
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
#simpan path dataset
path_data = "/content/drive/MyDrive/Tugas Akhir/dataset/new_data/dataset_new.csv"

#read dataset
data_dataset = pd.read_csv(path_data, sep=",", header=[0], encoding="UTF-8")
print(data_dataset)

                                                 ulasan   label
0     Terimakasih, pagi ini saya mengedit dan memper...    Baik
1     Video call lebih baik gambar jernih, walau kea...    Baik
2     Good..sangat berguna sekali. Semoga dapat meng...    Baik
3     tolong tambahkan fitur "otomatis keluar", fitu...    Baik
4     Baguss, tolong lebih di tingkatkan lagi. Biar ...    Baik
...                                                 ...     ...
1995  Biasanya pakai meet gampang, tapi dari sabtu k...  Kurang
1996  Tolong dah setiap APK Google jangan update lan...  Kurang
1997  Setelah update kok malah seperti ini sering se...  Kurang
1998  Gimana dah baru juga join tiba tiba mati,padah...  Kurang
1999  Lebih bgus yg dulu dri pd yg skrng,yang sekara...  Kurang

[2000 rows x 2 columns]


In [ ]:
#casefolding
def casefolding(ulasan):
  ulasan = ulasan.lower()
  ulasan = ulasan.strip(" ")
  ulasan = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|
\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))\n''', '', ulasan)
  return ulasan
data_dataset['ulasan'] = data_dataset['ulasan'].apply(casefolding)
data_dataset.head(10060)

,ulasan,label
0,"terimakasih, pagi ini saya mengedit dan memper...",Baik
1,"video call lebih baik gambar jernih, walau kea...",Baik
2,good..sangat berguna sekali. semoga dapat meng...,Baik
3,"tolong tambahkan fitur ""otomatis keluar"", fitu...",Baik
4,"baguss, tolong lebih di tingkatkan lagi. biar ...",Baik
...,...,...
1995,"biasanya pakai meet gampang, tapi dari sabtu k...",Kurang
1996,tolong dah setiap apk google jangan update lan...,Kurang
1997,setelah update kok malah seperti ini sering se...,Kurang
1998,"gimana dah baru juga join tiba tiba mati,padah...",Kurang


In [ ]:
# #casefolding
# data = data_casefolding.str.lower()
# data.head()

In [ ]:
#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|
\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))\n''',
 " ", tweet) for tweet in data_dataset['ulasan']]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  
#hapus simbol'[!#?,.:";@()-_/\']'
#  hapus tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]
data_filtering = pd.Series(filtering_numeric)

In [ ]:
data_filtering

0       terimakasih  pagi ini saya mengedit dan memper...
1       video call lebih baik gambar jernih  walau kea...
2       good  sangat berguna sekali  semoga dapat meng...
3       tolong tambahkan fitur  otomatis keluar   fitu...
4       baguss  tolong lebih di tingkatkan lagi  biar ...
                              ...                        
1995    biasanya pakai meet gampang  tapi dari sabtu k...
1996    tolong dah setiap apk google jangan update lan...
1997    setelah update kok malah seperti ini sering se...
1998    gimana dah baru juga join tiba tiba mati padah...
1999    lebih bgus yg dulu dri pd yg skrng yang sekara...
Length: 2000, dtype: object

In [ ]:
#tokenization
data_tokens = [word_tokenize(line) for line in data_filtering]
print(data_tokens)
data_dataset.head(2000)

[['terimakasih', 'pagi', 'ini', 'saya', 'mengedit', 'dan', 'memperbaharui', 'data', 'saya', 'krn', 'lupa', 'password', 'lama', 'ganti', 'password', 'baru', 'saya', 'harapkan', 'membuat', 'saya', 'lebih', 'lancar', 'menggunakan', 'akun', 'saya'], ['video', 'call', 'lebih', 'baik', 'gambar', 'jernih', 'walau', 'keadaan', 'pencahayaan', 'redup', 'tapi', 'masih', 'kelihatan', 'sangat', 'di', 'sayang', 'kan', 'battery', 'sangat', 'boros', 'klau', 'pakai', 'meet', 'untuk', 'video', 'call', 'tingkatkan', 'lagi'], ['good', 'sangat', 'berguna', 'sekali', 'semoga', 'dapat', 'mengupgrade', 'aplikasi', 'ini', 'dan', 'dapat', 'di', 'tingkatkan', 'lagi', 'aplikasi', 'ini', 'menjadi', 'lebih', 'baik', 'ke', 'depannya'], ['tolong', 'tambahkan', 'fitur', 'otomatis', 'keluar', 'fitur', 'tersebut', 'memudahkan', 'para', 'mahasiswa', 'yang', 'ketiduran', 'saat', 'presentasi', 'sampai', 'berakhirnya', 'presentasi', 'sehingga', 'mahasiswa', 'yang', 'tertidur', 'tersebut', 'tidak', 'berada', 'di', 'room', 'm

,ulasan,label
0,"terimakasih, pagi ini saya mengedit dan memper...",Baik
1,"video call lebih baik gambar jernih, walau kea...",Baik
2,good..sangat berguna sekali. semoga dapat meng...,Baik
3,"tolong tambahkan fitur ""otomatis keluar"", fitu...",Baik
4,"baguss, tolong lebih di tingkatkan lagi. biar ...",Baik
...,...,...
1995,"biasanya pakai meet gampang, tapi dari sabtu k...",Kurang
1996,tolong dah setiap apk google jangan update lan...,Kurang
1997,setelah update kok malah seperti ini sering se...,Kurang
1998,"gimana dah baru juga join tiba tiba mati,padah...",Kurang


In [ ]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
data_notstopword = [removeStopWords(line,stopwords) for line in data_tokens]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
print(pd.DataFrame(default_stop_words))

          0
0       ada
1    adalah
2    adanya
3    adapun
4      agak
..      ...
753    wong
754   yaitu
755   yakin
756   yakni
757    yang

[758 rows x 1 columns]


In [ ]:
len(data_notstopword)

2000

In [ ]:
data_notstopword

[['terimakasih',
  'pagi',
  'mengedit',
  'memperbaharui',
  'data',
  'krn',
  'lupa',
  'password',
  'ganti',
  'password',
  'harapkan',
  'lancar',
  'akun'],
 ['video',
  'call',
  'gambar',
  'jernih',
  'pencahayaan',
  'redup',
  'sayang',
  'battery',
  'boros',
  'klau',
  'pakai',
  'meet',
  'video',
  'call',
  'tingkatkan'],
 ['good',
  'berguna',
  'semoga',
  'mengupgrade',
  'aplikasi',
  'tingkatkan',
  'aplikasi',
  'depannya'],
 ['tolong',
  'tambahkan',
  'fitur',
  'otomatis',
  'fitur',
  'memudahkan',
  'mahasiswa',
  'ketiduran',
  'presentasi',
  'presentasi',
  'mahasiswa',
  'tertidur',
  'room',
  'meeting',
  'terima',
  'kasih'],
 ['baguss',
  'tolong',
  'tingkatkan',
  'biar',
  'screenshot',
  'orangnya',
  'kebawa',
  'layar',
  'zoom',
  'slide',
  'slide',
  'orangnya',
  'kebawa',
  'dikiranya',
  'mengikuti',
  'perkuliahan',
  'plisss'],
 ['gw',
  'sih',
  'salah',
  'meet',
  'app',
  'terenak',
  'android',
  'ringan',
  'ga',
  'ngelag',
  '

In [ ]:
factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
def stemmer(line):
    temp = list()
    for word in line:
      if(word not in data_dataset):
        word = ind_stemmer.stem(word)
      if(len(word)>3):
        temp.append(word)
    return temp

reviews = [stemmer (line) for line in data_notstopword]
print(reviews)

[['terimakasih', 'pagi', 'edit', 'baharu', 'data', 'lupa', 'password', 'ganti', 'password', 'harap', 'lancar', 'akun'], ['video', 'call', 'gambar', 'jernih', 'cahaya', 'redup', 'sayang', 'battery', 'boros', 'klau', 'pakai', 'meet', 'video', 'call', 'tingkat'], ['good', 'guna', 'moga', 'mengupgrade', 'aplikasi', 'tingkat', 'aplikasi', 'depan'], ['tolong', 'tambah', 'fitur', 'otomatis', 'fitur', 'mudah', 'mahasiswa', 'tidur', 'presentasi', 'presentasi', 'mahasiswa', 'tidur', 'room', 'meeting', 'terima', 'kasih'], ['baguss', 'tolong', 'tingkat', 'biar', 'screenshot', 'orang', 'bawa', 'layar', 'zoom', 'slide', 'slide', 'orang', 'bawa', 'kira', 'ikut', 'kuliah', 'plisss'], ['salah', 'meet', 'enak', 'android', 'ringan', 'ngelag', 'ngelag', 'kalo', 'jaring', 'jelek', 'doang', 'burik', 'gambar', 'audio', 'tetep', 'jernih', 'jaring', 'jelek', 'fitur', 'performa', 'keren', 'abis', 'update', 'nice', 'lahh', 'pokok'], ['aplikasi', 'sisi', 'resolusi', 'video', 'audio', 'guna', 'asing', 'kendala', '

In [ ]:
#Pembuatan Kamus kata
t  = Tokenizer()
fit_text = reviews
t.fit_on_texts(fit_text)

#Pembuatan Id masing-masing kata
sequences = t.texts_to_sequences(reviews)

#hapus duplikat kata yang muncul
list_set_sequence = [list(dict.fromkeys(seq)) for seq in sequences]

#mencari max length sequence
def FindMaxLength(lst): 
    maxList = max((x) for x in lst) 
    maxLength = max(len(x) for x in lst ) 
    return maxList, maxLength 
      
# Driver Code 
max_seq, max_length_seq = FindMaxLength(list_set_sequence)
jumlah_index = len(t.word_index) +1

print('jumlah index : ',jumlah_index,'\n')
print('word_index : ',t.word_index,'\n')
print('index kalimat asli     : ', sequences,'\n')
print('kalimat tanpa duplikat : ',list_set_sequence,'\n')
print('panjang max kalimat : ', max_length_seq,'kata','\n')
# print('kalimat terpanjang setelah dihapus duplikat : ', max_seq,'\n')

count_word = [len(i) for i in list_set_sequence]
print('list panjang kalimat : ', count_word)
max_len_word = max(count_word)
print(max_len_word)

jumlah index :  7169 

word_index :  {'saya': 1, 'ini': 2, 'bisa': 3, 'aplikasi': 4, 'bagus': 5, 'di': 6, 'sangat': 7, 'nya': 8, 'meet': 9, 'dan': 10, 'tidak': 11, 'keluar': 12, 'baik': 13, 'google': 14, 'untuk': 15, 'yang': 16, 'ajar': 17, 'tolong': 18, 'apk': 19, 'gak': 20, 'jadi': 21, 'bantu': 22, 'ada': 23, 'banget': 24, 'tapi': 25, 'lagi': 26, 'kenapa': 27, 'ya': 28, 'buat': 29, 'aku': 30, 'sendiri': 31, 'masuk': 32, 'guna': 33, 'suka': 34, 'ga': 35, 'kok': 36, 'download': 37, 'kasih': 38, 'padahal': 39, 'yg': 40, 'mau': 41, 'update': 42, 'lebih': 43, 'terus': 44, 'buka': 45, 'sekolah': 46, 'gk': 47, 'sekali': 48, 'dari': 49, 'kurang': 50, 'saat': 51, 'sama': 52, 'karena': 53, 'suara': 54, 'online': 55, 'jelek': 56, 'malah': 57, 'aja': 58, 'sudah': 59, 'guru': 60, 'juga': 61, 'hp': 62, 'dengan': 63, 'lama': 64, 'pas': 65, 'bintang': 66, 'pakai': 67, 'baru': 68, 'pake': 69, 'tiba': 70, 'sering': 71, 'daring': 72, 'ke': 73, 'sih': 74, 'udah': 75, 'kalo': 76, 'meeting': 77, 'banyak':

In [ ]:
min_len_word=min(count_word)
print(min_len_word)

0


In [ ]:
print ("Original list is : " + str(list_set_sequence[0])) 

Original list is : [42, 246, 8, 43, 1237, 6, 1362, 55, 68, 318, 213, 942, 32, 4, 2, 18, 3030, 1541, 13, 26, 15, 365, 138, 649, 569, 102, 38, 332, 66, 36, 76, 75, 579, 775]


In [ ]:
#Padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

padding= pad_sequences([list(list_set_sequence[i]) for i in range(len(list_set_sequence))],
maxlen= max_len_word, padding='pre')
padding[:10]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,   42,  246,    8,   43, 1237,    6, 1362,   55,   68,  318,
         213,  942,   32,    4,    2,   18, 3030, 1541,   13,   26,   15,
         365,  138,  649,  569,  102,   38,  332,   66,   36,   76,   75,
         579,  775],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,   50, 3031,   52,   40,
           6, 3032,  888,   71,  943, 3033,   13,   96,   92,  302,  144,
         295, 2190],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [ ]:
reviews2 = [" ".join(r) for r in reviews]

Featur Extraction

In [6]:
raw_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/dataset/dataclean_TA.csv', encoding='latin1')
raw_data

,ulasan,label
0,update nilai nya siakad online masuk aplikasi ...,Kurang
1,relate yg wab server down maintenance yg dong ...,Kurang
2,log in sso web bisa tp masuk apk email sso sed...,Kurang
3,pas buka mucul udh tunggu jam muncul hitam trus,Kurang
4,kembang apklikasi ios nya,Kurang
...,...,...
10055,mantap tahan,Baik
10056,bantu,Baik
10057,bantu,Baik
10058,maaf soal emosi tulis ini maaaaaaaaaaaaf bange...,Baik


In [ ]:
# train,test = train_test_split(raw_data, test_size = 0.2, random_state = 42)

In [7]:
raw_data.loc[[7, 10], 'ulasan']

7     ipk cantum aplikasi beda cantum web kecewa
10                          rip guna ios iphone 
Name: ulasan, dtype: object

In [8]:
X_train = raw_data['ulasan'].astype('U')
y_train = raw_data['label']

In [9]:
X_train.head(3)

0    update nilai nya siakad online masuk aplikasi ...
1    relate yg wab server down maintenance yg dong ...
2    log in sso web bisa tp masuk apk email sso sed...
Name: ulasan, dtype: object

In [10]:
y_train.head(3)

0    Kurang
1    Kurang
2    Kurang
Name: label, dtype: object

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

In [14]:
data_clean = raw_data.astype({'label' : 'category'})
data_clean = raw_data.astype({'ulasan' : 'string'})
data_clean.dtypes

# TF-IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
text_tf = tf.fit_transform(data_clean['ulasan'].astype('U'))
print(text_tf)

X_train, X_test, y_train, y_test = train_test_split(text_tf, data_clean['ulasan'], test_size=0.20, random_state = 42)

  (0, 2681)	0.15552399261874073
  (0, 6146)	0.14578861396958112
  (0, 6886)	0.09272625782990378
  (0, 3185)	0.0933345256974321
  (0, 1089)	0.09022007356643047
  (0, 4573)	0.1888609426227308
  (0, 3248)	0.07988847974575972
  (0, 6584)	0.09897577589429328
  (0, 855)	0.1485545046730966
  (0, 737)	0.14375147925358192
  (0, 6925)	0.17828000119673393
  (0, 5395)	0.1972595083163408
  (0, 6743)	0.06935777007192065
  (0, 2872)	0.09081876875930792
  (0, 459)	0.15009599487345945
  (0, 4092)	0.07787522340114175
  (0, 5006)	0.1708902842614727
  (0, 6070)	0.7554437704909231
  (0, 4875)	0.18157653442910163
  (0, 4801)	0.24843310229333404
  (0, 6962)	0.16611517819757882
  (1, 6027)	0.35473260566403964
  (1, 3300)	0.23883614213507842
  (1, 6446)	0.2316330899832525
  (1, 1907)	0.24851955819244578
  :	:
  (10051, 841)	0.32378820874212716
  (10052, 1031)	0.6541244328956185
  (10052, 130)	0.4992000070620193
  (10052, 2426)	0.3016836879727593
  (10052, 5493)	0.3977051298477439
  (10052, 6346)	0.271547714669

In [ ]:
# import re #casefolding
# import nltk #filtering
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# from sklearn.pipeline import Pipeline
# import nltk.corpus as corpus
# nltk.download('all')

In [12]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
import nltk.corpus as corpus
nltk.download('all')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [ ]:
# k_fold = KFold(n=len(corpus.data), n_folds=6)
# confusion = np.array([[0, 0], [0, 0]])

# pipeline = Pipeline([
#     ('vectorizer',  CountVectorizer(stop_words=stopwords, ngram_range=(1, 2))),
#     ('classifier',  MultinomialNB()) ])

# # also you are not using the indices...
# for train_indices, test_indices in k_fold:
#     pipeline.fit(corpus.data[train_indices], corpus.target[train_indices])
#     predictions = pipeline.predict(corpus.data[test_indices])

In [15]:
count_vect = CountVectorizer()
count_vect.fit(data_clean['ulasan'].astype('U'))
count_vect_transform = count_vect.transform(data_clean)

# X_train, X_test, y_train, y_test = train_test_split(count_vect_transform, y_train, test_size=0.20, random_state = 42)

In [16]:
count_vect_transform

<2x7334 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [17]:
count_vect_transform.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:
pd.DataFrame(count_vect_transform.toarray(), columns = count_vect.get_feature_names()).head()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,00gb,02,06,07,09,0rg,20,200,200mb,...,zelain,zom,zomm,zonk,zoom,zoomm,zoomnya,zoomsi,zooom,zzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# TF IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

tfidf_vect = TfidfTransformer()
X_train_tfidf = tfidf_vect.fit(count_vect_transform)
X_train_tfidf_transform = X_train_tfidf.transform(count_vect_transform)

In [20]:
pd.DataFrame(X_train_tfidf_transform.toarray(), columns = count_vect.get_feature_names()).head()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,00gb,02,06,07,09,0rg,20,200,200mb,...,zelain,zom,zomm,zonk,zoom,zoomm,zoomnya,zoomsi,zooom,zzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
clf = MultinomialNB().fit(X_train_tfidf_transform, y_train)

AttributeError: ignored

In [ ]:
print(clf.predict(count_vect.transform(['Kurang menarik dari segi tampilannya'])))

NameError: ignored

In [ ]:
# Save Model
import pickle
from pickle import dump
from pickle import load
from sklearn.metrics import accuracy_score

pickle.dump(count_vect,open('', 'wb'))
pickle.dump(clf, open('', 'wb'))